In [1]:
import pandas as pd
import numpy as np
import nltk
import random
import time
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from copy import copy, deepcopy
import collections
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from pandas.tseries.offsets import *
from sklearn.neighbors import NearestNeighbors
lmtzr=WordNetLemmatizer()

In [2]:
data_ads = "/Users/MaxTan/Documents/CU_16fall/BDA/Project/Data/"

user_raw = pd.read_csv(data_ads + "wow3_user2.csv", names = ["user_id", "review_count", "average_stars", 
                                               "friends", "fans", "votes_cool", "votes_funny", 
                                               "votes_useful", "num", "degree", "coefficient"])

business_raw = pd.read_csv(data_ads+"wow3_business_mysql.csv", names = ["business_id", "name", "latitude", 
                                                           "longitude", "stars", "categories", 
                                                           "review_count", "open"])

review_raw = pd.read_csv(data_ads+"wow3_review_mysql.csv", names = ["review_id", "user_id", "business_id", 
                                                       "stars", "text", "date", "votes_funny", 
                                                       "votes_useful"])

all_raw = pd.read_csv(data_ads+"wow3_all2.csv", names = ["business_id", 
                                                       "name", "stars_business","business_review_count",
                                                       "categories", "open", "review_id", 
                                                       "stars_review", "text", "date", "votes_funny", 
                                                       "votes_useful","user_id", "user_review_count", 
                                                       "average_stars","friends", "fans", "num", 
                                                       "degree", "coefficient"])


/Users/MaxTan/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#1. Near Neighbors for recommendation by different types:

In [4]:
#Input:
longitude = -80.0089
latitude = 40.3719
num_recommend = 6
typ ='Restaurants'

In [5]:
weight =100
bus_gb_df = deepcopy(all_raw)
#Group by different businesses for different rates and replace them by their means:
bus_df = bus_gb_df.groupby(['business_id']).mean()

bus_df['business_id'] = bus_df.index
bus_df.index = range(len(bus_df))
bus = list(set(bus_df['business_id']))
ind = [True if i in bus else False for i in business_raw['business_id']]
business_revised = business_raw[ind]

#Merge two tables:
knn_df = pd.merge(bus_df,business_revised,on='business_id')

#Only choose open business for recommendation:
knn_df_open = knn_df[knn_df['open']=="T"]

#Filter the type of business:
ind = [True if typ in i else False for i in knn_df_open.categories.values]
knn_df_open_typ = knn_df_open[ind]

#Set training data:
trainX = knn_df_open_typ[['stars_business','stars_review','average_stars','latitude','longitude']]
trainX['latitude'] = trainX['latitude'].map(lambda x:x*weight)
trainX['longitude'] = trainX['longitude'].map(lambda x:x*weight)
longitude *= weight
latitude *= weight

#Set test data:
testX = [5,5,5,latitude,longitude]

/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
neigh = NearestNeighbors(n_neighbors=num_recommend)
neigh.fit(trainX)
#Set parameters for the Nearest Neighbors algorithms:
NearestNeighbors(algorithm='auto', leaf_size=30)
result = neigh.kneighbors(testX)

In [8]:
#Output:
#knn_df_open_typ.iloc[result[1][0]]

stars_business  business_review_count  stars_review  votes_funny  \
2404             4.0                    9.0      3.833333     0.000000   
2959             4.5                   26.0      4.500000     0.041667   
1533             4.5                    3.0      5.000000     0.000000   
475              4.0                   20.0      3.941176     0.529412   
3456             4.5                   27.0      4.346154     0.384615   
2250             3.5                    3.0      4.000000     0.000000   

      votes_useful  user_review_count  average_stars      fans           num  \
2404      1.666667         116.833333       3.815000  1.833333   5411.666667   
2959      0.375000          63.500000       3.879167  3.458333  11524.708333   
1533      1.000000          14.000000       4.270000  0.000000   4118.000000   
475       0.764706          55.470588       3.705294  1.823529   4430.529412   
3456      1.230769          59.653846       3.736538  3.307692  16975.961538   
2250      1.000000          29.000000       3.620000  1.000000    970.000000   

         degree  coefficient             business_id  \
2404  14.666667     0.154708  e5HRYpEIkyrKNR6yA2rnqQ   
2959  91.875000     0.048626  nrCa99QfoXdTB7dzwTQqIQ   
1533   2.000000     0.000000  PscxgmrteHI-6qdI-vIm5g   
475   16.882353     0.074838  7UBqeUvQeZd5o2Jw9F8r0g   
3456  84.115385     0.046056  wwmMJIDyq7QHBzb9YSaCkQ   
2250   6.000000     0.000000  bg1e__l7Jh3oDhasW5KFvw   

                            name  latitude  longitude  stars  \
2404         Trolley Barn Eatery   40.3682   -80.0128    4.0   
2959               Kabob-G Grill   40.3655   -80.0256    4.5   
1533            Boulevard Lounge   40.3920   -80.0140    4.5   
475            The Original Gyro   40.3652   -80.0181    4.0   
3456  Franco's Pizzeria Italiana   40.3658   -80.0251    4.5   
2250                 Burger King   40.3698   -80.0117    3.5   

                                             categories  review_count open  
2404          [Breakfast & Brunch, Diners, Restaurants]             9    T  
2959  [Middle Eastern, Lebanese, Mediterranean, Rest...            26    T  
1533  [Bars, American (Traditional), Nightlife, Rest...             3    T  
475                                [Greek, Restaurants]            20    T  
3456                               [Pizza, Restaurants]            27    T  
2250                  [Burgers, Fast Food, Restaurants]             3    T